In [1]:
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

# Path to your Roboflow annotation files (assuming in JSON format like COCO keypoints)
ANNOTATIONS_DIR = 'benchpress_annotations/'  # Adjust this path
OUTPUT_CSV = 'joint_angle_dataset.csv'

# Define the keypoints of interest
KEYPOINT_NAMES = ['shoulder', 'elbow', 'wrist']
KEYPOINT_INDEX = {
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10
}

def calculate_angle(a, b, c):
    """Returns the angle at point b given three points a-b-c."""
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))  # Clip for numerical stability
    return np.degrees(angle)

def process_json_file(json_file):
    """Process a single annotation JSON file and return angle data."""
    with open(json_file, 'r') as f:
        data = json.load(f)

    entries = []
    for ann in data['annotations']:
        if 'keypoints' not in ann or len(ann['keypoints']) < 33:
            continue  # Skip malformed data

        kp = np.array(ann['keypoints']).reshape(-1, 3)
        sample = {}

        for side in ['left', 'right']:
            shoulder = kp[KEYPOINT_INDEX[f'{side}_shoulder']][:2]
            elbow = kp[KEYPOINT_INDEX[f'{side}_elbow']][:2]
            wrist = kp[KEYPOINT_INDEX[f'{side}_wrist']][:2]

            # Angle at shoulder (between torso -> shoulder -> elbow)
            angle_shoulder = calculate_angle(wrist, shoulder, elbow)

            # Angle at elbow (between shoulder -> elbow -> wrist)
            angle_elbow = calculate_angle(shoulder, elbow, wrist)

            sample[f'{side}_shoulder_angle'] = angle_shoulder
            sample[f'{side}_elbow_angle'] = angle_elbow

        entries.append(sample)
    return entries

def preprocess_dataset():
    all_samples = []
    files = [f for f in os.listdir(ANNOTATIONS_DIR) if f.endswith('.json')]

    for file in tqdm(files, desc="Processing files"):
        path = os.path.join(ANNOTATIONS_DIR, file)
        all_samples.extend(process_json_file(path))

    df = pd.DataFrame(all_samples)
    df.dropna(inplace=True)
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"[✓] Processed data saved to {OUTPUT_CSV}")

if __name__ == "__main__":
    preprocess_dataset()


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'benchpress_annotations/'